In [1]:
# Experiment 2 Script for COVID-19 Forecasting

# Change path to the root directory of the project
import os

os.chdir("../../")

# Description: This script contains the code for the second experiment in the project,
# forecasting COVID-19 MVBeds using various RNN models and hyperparameter tuning with Simulated Annealing.

# Imports for handling data
import shutil
import numpy as np
import pandas as pd
from pathlib import Path
from itertools import cycle

# Imports for machine learning
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

from sklearn.metrics import (
    mean_absolute_error as mae,
    mean_squared_error as mse,
    mean_absolute_percentage_error as mape,
)

# Imports for visualization
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# Progress bar
from tqdm.autonotebook import tqdm

tqdm.pandas()

# Local imports for data loaders and models
from src.utils import plotting_utils
from src.dl.dataloaders import TimeSeriesDataModule
from src.dl.multivariate_models import (
    SingleStepRNNConfig,
    SingleStepRNNModel,
    Seq2SeqConfig,
    Seq2SeqModel,
    RNNConfig,
)
from src.transforms.target_transformations import AutoStationaryTransformer

# Set seeds for reproducibility
pl.seed_everything(42)
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Set float32 matrix multiplication precision
torch.set_float32_matmul_precision("high")

# Set default Plotly template
pio.templates.default = "plotly_white"

# Ignore warnings
import warnings

warnings.filterwarnings("ignore")

# Set logging configuration
import logging

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

Seed set to 42


In [2]:
# Utility Functions


def format_plot(
    fig, legends=None, xlabel="Time", ylabel="Value", title="", font_size=15
):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t: t.update(name=next(names)))
    fig.update_layout(
        autosize=False,
        width=900,
        height=500,
        title_text=title,
        title={"x": 0.5, "xanchor": "center", "yanchor": "top"},
        titlefont={"size": 20},
        legend_title=None,
        legend=dict(
            font=dict(size=font_size),
            orientation="h",
            yanchor="bottom",
            y=0.98,
            xanchor="right",
            x=1,
        ),
        yaxis=dict(
            title_text=ylabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
        xaxis=dict(
            title_text=xlabel,
            titlefont=dict(size=font_size),
            tickfont=dict(size=font_size),
        ),
    )
    return fig


def mase(actual, predicted, insample_actual):
    mae_insample = np.mean(np.abs(np.diff(insample_actual)))
    mae_outsample = np.mean(np.abs(actual - predicted))
    return mae_outsample / mae_insample


def forecast_bias(actual, predicted):
    return np.mean(predicted - actual)


def rmse(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))

def nrmse(y_true, y_pred):
    """Calculate the Normalized Root Mean Square Error (NRMSE)."""
    return np.sqrt(mse(y_true, y_pred)) / (
        np.max(y_true) - np.min(y_true)
    )


def plot_forecast(
    pred_df,
    forecast_columns,
    selected_area,
    forecast_display_names=None,
    save_path=None,
):
    if forecast_display_names is None:
        forecast_display_names = forecast_columns
    else:
        assert len(forecast_columns) == len(forecast_display_names)

    mask = ~pred_df[forecast_columns[0]].isnull()
    colors = px.colors.qualitative.D3  # Use a colorblind-friendly palette
    act_color = colors[0]
    colors = cycle(colors[1:])

    fig = go.Figure()

    # Actual data plot
    fig.add_trace(
        go.Scatter(
            x=pred_df[mask].index,
            y=pred_df[mask].covidOccupiedMVBeds,
            mode="lines+markers",
            marker=dict(size=8, opacity=0.7, symbol="circle"),
            line=dict(color=act_color, width=3),
            name="Actual COVID-19 MVBeds trends",
        )
    )

    # Predicted data plot
    line_styles = ["solid", "dash", "dot", "dashdot"]
    markers = ["circle", "square", "diamond", "cross", "x", "triangle-up"]
    for col, display_col, line_style, marker in zip(
        forecast_columns, forecast_display_names, cycle(line_styles), cycle(markers)
    ):
        fig.add_trace(
            go.Scatter(
                x=pred_df[mask].index,
                y=pred_df.loc[mask, col],
                mode="lines+markers",
                marker=dict(size=6, symbol=marker),
                line=dict(color=next(colors), width=2, dash=line_style),
                name=display_col,
            )
        )

    fig.update_layout(
        title=f"COVID-19 MVBeds Forecast Comparison for {selected_area}",
        title_font=dict(size=20),
        xaxis_title="Date",
        yaxis_title="COVID-19 MVBeds",
        xaxis=dict(title_font=dict(size=15), tickfont=dict(size=12)),
        yaxis=dict(title_font=dict(size=15), tickfont=dict(size=12)),
        legend=dict(
            font=dict(size=10),
            orientation="h",
            yanchor="top",
            y=1.02,  # Move the legend down to separate it from the title
            xanchor="center",
            x=0.5,
        ),
        template="plotly_white",
        plot_bgcolor="rgba(0,0,0,0)",
        margin=dict(l=10, r=40, t=80, b=40),  # Adjust left margin for better alignment
        width=1200,
        height=600,
    )

    if save_path:
        pio.write_image(fig, save_path)
    return fig


def highlight_abs_min(s, props=""):
    return np.where(s == np.nanmin(np.abs(s.values)), props, "")


# Function to convert data to float32
def to_float32(df):
    for col in df.columns:
        df[col] = df[col].astype("float32")
    return df

In [3]:
# Load and Prepare Data
data_path = Path("data/processed/england_data.csv")
# data = pd.read_csv(data_path).drop("Unnamed: 0", axis=1)
data = pd.read_csv(data_path)
data["date"] = pd.to_datetime(data["date"])

# Select and Process Data
selected_area = "England"
# data_filtered = data[data["areaName"] == selected_area]
data.head()

,date,covidOccupiedMVBeds,cumAdmissions,hospitalCases,newAdmissions,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,population,openstreetmap_id,latitude,longitude,areaName
0,2020-04-01,0.0,23332,12059.0,3099,3989.0,694.0,35571.0,4730.0,56171302,1275584,474.050001,-12.133333,England
1,2020-04-02,1494.0,26264,12135.0,2932,3895.0,725.0,39466.0,5455.0,56171302,1275584,474.050001,-12.133333,England
2,2020-04-03,1788.0,28828,13635.0,2564,3878.0,737.0,43344.0,6192.0,56171302,1275584,474.050001,-12.133333,England
3,2020-04-04,1950.0,31421,15469.0,2593,3260.0,828.0,46604.0,7020.0,56171302,1275584,474.050001,-12.133333,England
4,2020-04-05,2097.0,34013,16657.0,2592,2994.0,823.0,49598.0,7843.0,56171302,1275584,474.050001,-12.133333,England


In [4]:
# Add time-lagged features
def add_lags(data, lags, features):
    added_features = []
    for feature in features:
        for lag in lags:
            new_feature = feature + f"_lag_{lag}"
            data[new_feature] = data[feature].shift(lag)
            added_features.append(new_feature)
    return data, added_features


lags = [1, 2, 3, 5, 7, 14, 21]
data, added_features = add_lags(data, lags, ["covidOccupiedMVBeds"])
data.dropna(inplace=True)


# Create temporal features
def create_temporal_features(df, date_column):
    df["month"] = df[date_column].dt.month
    df["day"] = df[date_column].dt.day
    df["day_of_week"] = df[date_column].dt.dayofweek
    return df


data = create_temporal_features(data, "date")
data = data.set_index("date")
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 874 entries, 2020-04-22 to 2022-09-12
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   covidOccupiedMVBeds         874 non-null    float64
 1   cumAdmissions               874 non-null    int64  
 2   hospitalCases               874 non-null    float64
 3   newAdmissions               874 non-null    int64  
 4   new_confirmed               874 non-null    float64
 5   new_deceased                874 non-null    float64
 6   cumulative_confirmed        874 non-null    float64
 7   cumulative_deceased         874 non-null    float64
 8   population                  874 non-null    int64  
 9   openstreetmap_id            874 non-null    int64  
 10  latitude                    874 non-null    float64
 11  longitude                   874 non-null    float64
 12  areaName                    874 non-null    object 
 13  covidOccupiedMVB

In [5]:
# Add rolling features
def add_rolling_features(df, window_size, columns, agg_funcs=None):
    if agg_funcs is None:
        agg_funcs = ["mean", "std", "min", "max"]
    added_features = {}
    for column in columns:
        for func in agg_funcs:
            roll_col_name = f"{column}_rolling_{window_size}_{func}"
            df[roll_col_name] = df[column].rolling(window_size).agg(func)
            if column not in added_features:
                added_features[column] = []
            added_features[column].append(roll_col_name)
    df.dropna(inplace=True)
    return df, added_features


window_size = 7
columns_to_roll = [
    "hospitalCases",
    "newAdmissions",
    "new_confirmed",
    "new_deceased",
    # "covidOccupiedMVBeds",
    # "susceptible",
    # "active_cases",
    # "recovered",
    # "cumulative_deceased",
]
agg_funcs = ["mean"]

merged_data, added_features = add_rolling_features(
    data, window_size, columns_to_roll, agg_funcs
)

for column, features in added_features.items():
    logging.info(f"{column}: {', '.join(features)}")

2024-06-11 14:33:40,352 - INFO - hospitalCases: hospitalCases_rolling_7_mean
2024-06-11 14:33:40,353 - INFO - newAdmissions: newAdmissions_rolling_7_mean
2024-06-11 14:33:40,355 - INFO - new_confirmed: new_confirmed_rolling_7_mean
2024-06-11 14:33:40,356 - INFO - new_deceased: new_deceased_rolling_7_mean


In [6]:
# plot cumulative_deceased
fig = px.line(
    merged_data,
    x=merged_data.index,
    y="hospitalCases_rolling_7_mean",
    title="Hospital Cases Rolling Mean",
)
fig.show()
fig.write_image("reports/figures/hospital_cases_rolling_mean.png")

In [7]:
# Filter data between the specified dates
start_date = "2020-04-27"
end_date = "2022-12-31"

min_date = merged_data.index.min()
max_date = merged_data.index.max()

# Calculate the range of dates
date_range = max_date - min_date
logging.info(f"Data ranges from {min_date} to {max_date} ({date_range.days} days)")

2024-06-11 14:33:45,550 - INFO - Data ranges from 2020-04-28 00:00:00 to 2022-09-12 00:00:00 (867 days)


In [8]:
# drop columns that are not needed
merged_data.drop(
    columns=[
        "areaName",
        "population",
        "openstreetmap_id",
        "latitude",
        "longitude",
    ],
    inplace=True,
)

In [9]:
# Set the target variable
target = "covidOccupiedMVBeds"

seasonal_period = 7
auto_stationary = AutoStationaryTransformer(seasonal_period=seasonal_period)

# Fit and transform the target column to make it stationary
data_stat = auto_stationary.fit_transform(merged_data[[target]], freq="D")

# Replace the original target values with the transformed stationary values
merged_data[target] = data_stat.values

# Print the transformed data to check
merged_data.head()

,covidOccupiedMVBeds,cumAdmissions,hospitalCases,newAdmissions,new_confirmed,new_deceased,cumulative_confirmed,cumulative_deceased,covidOccupiedMVBeds_lag_1,covidOccupiedMVBeds_lag_2,...,covidOccupiedMVBeds_lag_7,covidOccupiedMVBeds_lag_14,covidOccupiedMVBeds_lag_21,month,day,day_of_week,hospitalCases_rolling_7_mean,newAdmissions_rolling_7_mean,new_confirmed_rolling_7_mean,new_deceased_rolling_7_mean
date,,,,,,,,,,,,,,,,,,,,,
2020-04-28,43.598269,73874,13565.0,1296,3902.0,496.0,135248.0,24391.0,2194.0,2309.0,...,2701.0,2849.0,2472.0,4,28,1,14608.285714,1257.714286,3900.857143,563.428571
2020-04-29,43.174504,75169,13224.0,1295,4610.0,495.0,139858.0,24886.0,2093.0,2194.0,...,2646.0,2847.0,2561.0,4,29,2,14209.142857,1255.571429,3895.142857,542.714286
2020-04-30,42.688971,76396,12922.0,1227,4231.0,483.0,144089.0,25369.0,2045.0,2093.0,...,2563.0,2838.0,2546.0,4,30,3,13854.857143,1246.714286,3867.714286,526.714286
2020-05-01,42.353640,77477,12623.0,1081,4024.0,475.0,148113.0,25844.0,1991.0,2045.0,...,2414.0,2780.0,2820.0,5,1,4,13566.428571,1218.714286,3848.285714,508.714286
2020-05-02,41.316072,78457,12314.0,980,2677.0,424.0,150790.0,26268.0,1954.0,1991.0,...,2405.0,2868.0,2865.0,5,2,5,13296.714286,1204.428571,3768.857143,490.571429


In [10]:
# Calculate split points
total_days = date_range.days
train_end = min_date + pd.Timedelta(days=int(total_days * 0.70))
val_end = train_end + pd.Timedelta(days=int(total_days * 0.20))

# Split the data into training, validation, and testing sets
train = merged_data[merged_data.index <= train_end]
val = merged_data[(merged_data.index > train_end) & (merged_data.index <= val_end)]
test = merged_data[merged_data.index > val_end]

# Calculate the percentage of dates in each dataset
total_sample = len(merged_data)
train_sample = len(train) / total_sample * 100
val_sample = len(val) / total_sample * 100
test_sample = len(test) / total_sample * 100

print(
    f"Train: {train_sample:.2f}%, Validation: {val_sample:.2f}%, Test: {test_sample:.2f}%"
)
print(
    f"Train: {len(train)} samples, Validation: {len(val)} samples, Test: {len(test)} samples"
)
print(
    f"Max date in train: {train.index.max()}, Min date in validation: {val.index.min()}, Max date in test: {test.index.max()}"
)

Train: 69.93%, Validation: 19.93%, Test: 10.14%
Train: 607 samples, Validation: 173 samples, Test: 88 samples
Max date in train: 2021-12-25 00:00:00, Min date in validation: 2021-12-26 00:00:00, Max date in test: 2022-09-12 00:00:00


In [11]:
train.columns

Index(['covidOccupiedMVBeds', 'cumAdmissions', 'hospitalCases',
       'newAdmissions', 'new_confirmed', 'new_deceased',
       'cumulative_confirmed', 'cumulative_deceased',
       'covidOccupiedMVBeds_lag_1', 'covidOccupiedMVBeds_lag_2',
       'covidOccupiedMVBeds_lag_3', 'covidOccupiedMVBeds_lag_5',
       'covidOccupiedMVBeds_lag_7', 'covidOccupiedMVBeds_lag_14',
       'covidOccupiedMVBeds_lag_21', 'month', 'day', 'day_of_week',
       'hospitalCases_rolling_7_mean', 'newAdmissions_rolling_7_mean',
       'new_confirmed_rolling_7_mean', 'new_deceased_rolling_7_mean'],
      dtype='object')

In [12]:
# Concatenate the DataFrames
sample_df = pd.concat([train, val, test])

# Convert all the feature columns to float32
sample_df = to_float32(sample_df)

pinn_features = [
    "covidOccupiedMVBeds",
    "covidOccupiedMVBeds_lag_1",
    "covidOccupiedMVBeds_lag_2",
    "covidOccupiedMVBeds_lag_3",
    "covidOccupiedMVBeds_lag_5",
    "covidOccupiedMVBeds_lag_7",
    "covidOccupiedMVBeds_lag_14",
    "covidOccupiedMVBeds_lag_21",
    "month",
    "day",
    "day_of_week",
    "hospitalCases_rolling_7_mean",
    "newAdmissions_rolling_7_mean",
]

sample_df = sample_df[pinn_features]
cols = list(sample_df.columns)
cols.remove("covidOccupiedMVBeds")
sample_df = sample_df[cols + ["covidOccupiedMVBeds"]]
sample_df.head()

,covidOccupiedMVBeds_lag_1,covidOccupiedMVBeds_lag_2,covidOccupiedMVBeds_lag_3,covidOccupiedMVBeds_lag_5,covidOccupiedMVBeds_lag_7,covidOccupiedMVBeds_lag_14,covidOccupiedMVBeds_lag_21,month,day,day_of_week,hospitalCases_rolling_7_mean,newAdmissions_rolling_7_mean,covidOccupiedMVBeds
date,,,,,,,,,,,,,
2020-04-28,2194.0,2309.0,2405.0,2563.0,2701.0,2849.0,2472.0,4.0,28.0,1.0,14608.286133,1257.714233,43.598270
2020-04-29,2093.0,2194.0,2309.0,2414.0,2646.0,2847.0,2561.0,4.0,29.0,2.0,14209.142578,1255.571411,43.174503
2020-04-30,2045.0,2093.0,2194.0,2405.0,2563.0,2838.0,2546.0,4.0,30.0,3.0,13854.857422,1246.714233,42.688972
2020-05-01,1991.0,2045.0,2093.0,2309.0,2414.0,2780.0,2820.0,5.0,1.0,4.0,13566.428711,1218.714233,42.353642
2020-05-02,1954.0,1991.0,2045.0,2194.0,2405.0,2868.0,2865.0,5.0,2.0,5.0,13296.713867,1204.428589,41.316071


In [13]:
# Prepare DataModule for PyTorch Lightning
datamodule = TimeSeriesDataModule(
    data=sample_df,
    n_val=val.shape[0],
    n_test=test.shape[0],
    window=7,  # 7 days window
    horizon=1,  # single step
    normalize="global",  # normalizing the data
    batch_size=32,
    num_workers=0,
)
datamodule.setup()

# Initialize pred_df
pred_df = test.copy()

In [14]:
# Function to train and evaluate models
def train_and_evaluate(model, rnn_config, datamodule, test, train, algorithm_name, metric_record, pred_df):
    trainer = pl.Trainer(
        min_epochs=5,
        max_epochs=200,
        accelerator="gpu",
        devices=1,
        callbacks=[EarlyStopping(monitor="valid_loss", patience=20)],
    )
    trainer.fit(model, datamodule)

    predictions = trainer.predict(model, datamodule.test_dataloader())
    predictions = torch.cat(predictions).squeeze().detach().numpy()
    predictions = predictions * datamodule.train.std + datamodule.train.mean
    actuals = test["covidOccupiedMVBeds"].values

    assert actuals.shape == predictions.shape, "Mismatch in shapes between actuals and predictions"

    metrics = {
        "Algorithm": algorithm_name,
        "MAE": mae(actuals, predictions),
        "MAPE": mape(actuals, predictions),
        "RMSE": rmse(actuals, predictions),
        "NRMSE": nrmse(actuals, predictions),
        "MSE": mse(actuals, predictions),
        "MASE": mase(actuals, predictions, train["covidOccupiedMVBeds"].values),
        "Forecast Bias": forecast_bias(actuals, predictions),
    }

    value_formats = ["{}", "{:.4f}", "{:.4f}", "{:.4f}", "{:.4f}", "{:.4f}", "{:.4f}", "{:.2f}"]
    metrics = {
        key: format_.format(value)
        for key, value, format_ in zip(metrics.keys(), metrics.values(), value_formats)
    }

    pred_df_ = pd.DataFrame({algorithm_name: predictions}, index=test.index)
    pred_df = pred_df.join(pred_df_)
    metric_record.append(metrics)
    logging.info(metrics)
    return metric_record, pred_df

In [15]:
# Train Vanilla RNN
rnn_config = SingleStepRNNConfig(
    rnn_type="RNN",
    input_size=len(pinn_features),
    hidden_size=32,
    num_layers=5,
    bidirectional=False,
    learning_rate=1e-3,
)
model = SingleStepRNNModel(rnn_config)
algorithm_name = "Vanilla RNN"
metric_record, pred_df = train_and_evaluate(model, rnn_config, datamodule, test, train, algorithm_name, metric_record=[], pred_df=pred_df)

# Train Vanilla LSTM
rnn_config = SingleStepRNNConfig(
    rnn_type="LSTM",
    input_size=len(pinn_features),
    hidden_size=32,
    num_layers=5,
    bidirectional=True,
    learning_rate=1e-3,
)
model = SingleStepRNNModel(rnn_config)
algorithm_name = "Vanilla LSTM"
metric_record, pred_df = train_and_evaluate(model, rnn_config, datamodule, test, train, algorithm_name, metric_record, pred_df)

# Train Vanilla GRU
rnn_config = SingleStepRNNConfig(
    rnn_type="GRU",
    input_size=len(pinn_features),
    hidden_size=32,
    num_layers=5,
    bidirectional=True,
    learning_rate=1e-3,
)
model = SingleStepRNNModel(rnn_config)
algorithm_name = "Vanilla GRU"
metric_record, pred_df = train_and_evaluate(model, rnn_config, datamodule, test, train, algorithm_name, metric_record, pred_df)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type    | Params
---------------------------------
0 | rnn  | RNN     | 10.0 K
1 | fc   | Linear  | 33    
2 | loss | MSELoss | 0     
---------------------------------
10.0 K    Trainable params
0         Non-trainable params
10.0 K    Total params
0.040     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

2024-06-11 14:34:07,755 - INFO - {'Algorithm': 'Vanilla GRU', 'MAE': '9.1034', 'MAPE': '0.3152', 'RMSE': '10.2792', 'NRMSE': '2.7678', 'MSE': '105.6618', 'MASE': '24.7493', 'Forecast Bias': '-9.10'}


In [16]:
metric_record

[{'Algorithm': 'Vanilla RNN',
  'MAE': '7.1198',
  'MAPE': '0.2461',
  'RMSE': '8.0128',
  'NRMSE': '2.1576',
  'MSE': '64.2049',
  'MASE': '19.3566',
  'Forecast Bias': '-7.12'},
 {'Algorithm': 'Vanilla LSTM',
  'MAE': '3.1289',
  'MAPE': '0.1070',
  'RMSE': '3.5719',
  'NRMSE': '0.9618',
  'MSE': '12.7582',
  'MASE': '8.5066',
  'Forecast Bias': '0.08'},
 {'Algorithm': 'Vanilla GRU',
  'MAE': '9.1034',
  'MAPE': '0.3152',
  'RMSE': '10.2792',
  'NRMSE': '2.7678',
  'MSE': '105.6618',
  'MASE': '24.7493',
  'Forecast Bias': '-9.10'}]

In [17]:
# plot the forecast
forecast_columns = ["Vanilla RNN", "Vanilla LSTM", "Vanilla GRU"]
forecast_display_names = ["Vanilla RNN", "Vanilla LSTM", "Vanilla GRU"]
fig = plot_forecast(
    pred_df, forecast_columns, selected_area, forecast_display_names=forecast_display_names
)
fig.write_image("reports/figures/vanilla_rnn_lstm_gru_forecast.pdf")
fig.show()